# XML einlesen

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import os as os

def read_xml_file(file_path):
  """Liest eine XML-Datei ein und gibt den Wurzelknoten zurück."""
  try:
    print("file_path: ", file_path)
    tree = ET.parse(file_path)
    root = tree.getroot()
    return root
  except FileNotFoundError:
    print(f"Datei nicht gefunden: {file_path}")
    return None
  except ET.ParseError:
    print(f"Fehler beim Parsen der XML-Datei: {file_path}")
    return None


In [ ]:
# Beispiel: XML-Datei aus dem aktuellen Verzeichnis einlesen
# Ersetzen Sie 'your_xml_file.xml' durch den tatsächlichen Dateinamen
xml_file_name = '9151-21028044-000_.xml'

current_directory = os.getcwd()
xml_file_path = os.path.join(current_directory, xml_file_name)

root = read_xml_file(xml_file_path)

if root is not None:
  # Hier können Sie mit den XML-Daten arbeiten
  print("XML-Datei erfolgreich eingelesen!")
  # Beispiel: Alle Elemente im XML-Baum ausgeben
  for child in root:
    print(child.tag, child.attrib)


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Read XML File

In [ ]:
# 9151-20201689-000_.xml ohne KLB_DATEN
# 9151-00000501-000_.xml mit KLB_DATEN und Makrokontur
# 9151-00000502-000_.xml Makrokontur 4Nutes
# 9151-21028044-000_.xml Makrokontur 5Nutes
# 9151-20204659-000_.xml bis groove4 / !
# 9151-21001084-000_.xml bis groove4 / ok
# 9151-21001708-000_.xml bis groove3 / !
# 9151-21014753-000_.xml bis groove4 / !
# 9151-21018270-000_.xml bis groove4 / !
# 9151-21022736-000_.xml bis groove3 / ok
# 9151-01000972-000_.xml ohne Makro-Contur
# 9151-21043089-000_.xml bis groove3 / ok

print(uploaded)
print(fn)
root = read_xml_file(fn)



# Read the Macrocontur

In [ ]:
def readMacrocontur(root):
  klb_data_found = False
  yklb_values = []
  xklb_values = []
  if root:
    for element in root.findall(".//MAKROKONTUR"):
      for child in element:
        if child.tag == 'KLB_DATEN':
          for child2 in child:
            if child2.tag == 'ROW':
              yklb_found = False
              xklb_found = False
              for child3 in child2:
                if child3.tag == 'YKLB':
                  yklb_values.append(float(child3.text))
                  yklb_found = True
                elif child3.tag == 'XKLB':
                  xklb_values.append(float(child3.text))
                  xklb_found = True
              if not yklb_found:
                yklb_values.append(np.nan)  # Füge NaN hinzu, falls YKLB fehlt
              if not xklb_found:
                xklb_values.append(np.nan)  # Füge NaN hinzu, falls XKLB fehlt
  yklb_array = np.array(yklb_values)
  xklb_array = np.array(xklb_values)
  return xklb_array, yklb_array



In [ ]:
'''
<FKA_TAB COUNT="8">
<FKA>
<LINE TYP2="23" ID="FD_LINE" COLOR="FD_COLOR_BLUE" LINETHICK="FD_LINETHICK_NORMAL" LINESTYLE="FD_LINETYPE_SOLID" LAYER="FD_LAYER_1">
<TYPEID>23</TYPEID>
<TYPENAME>NA</TYPENAME>
<XA>61</XA>
<YA>0</YA>
<XE>264</XE>
<YE>0</YE>
</LINE>
<ABSCHNITT>
<NUMMER>1</NUMMER>
</ABSCHNITT>
<HOEHE>0.000</HOEHE>
'''
def readProfileLimits(root):
  level_values = []
  level_number_values = []
  if root:
    #print("Root")
    for element in root.findall(".//FKA_TAB"):
      #print("FKA_TAB")
      for child in element:
        #print(child.tag )
        for child2 in child:
          #print(child2.tag )
          if child2.tag == 'HOEHE':
              level_values.append(float(child2.text))
          if child2.tag == 'ABSCHNITT':
            #print(child2.tag )
            for child3 in child2:
              #print(child3.tag )
              if child3.tag == 'NUMMER':
                level_number_values.append(float(child3.text))
  return np.array(level_values), np.array(level_number_values)

level_limit, level_limit_number = readProfileLimits(root)
print(level_limit)
print(level_limit_number)



In [ ]:
def readGeometrieProfileCurves(root):
  xa  = []
  ya  = []
  xe  = []
  ye  = []
  level_number_values = []
  if root:
    print("Root")
    for element in root.findall(".//GEOMETRIE_FORMKURVEN"):
      print("GEOMETRIE_FORMKURVEN")
      for child in element:
        if child.tag == 'LINE':
          for child2 in child:
            if child2.tag == 'XA':
              xa.append(float(child2.text))
            if child2.tag == 'YA':
              ya.append(float(child2.text))
            if child2.tag == 'XE':
              xe.append(float(child2.text))
            if child2.tag == 'YE':
              ye.append(float(child2.text))

  return np.array(xa), np.array(ya),np.array(xe), np.array(ye)

xa, ya, xe, ye = readGeometrieProfileCurves(root)
print(xa)
print(ya)


# Correction X/Y Coordinates for Macrocontur

In [ ]:

# Reihenfolge der letzten zwei einträge im Array tauschen
def changeContur(xarray,yarray, nxklb, nyklb):
  print("changeContur")
  x = xarray[nxklb-1]
  y = yarray[nyklb-1]

  xarray[nxklb-1] = xarray[nxklb]
  yarray[nyklb-1] = yarray[nyklb]

  xarray[nxklb] = x
  yarray[nxklb] = y
  return xarray, yarray


In [ ]:
print("Read KLB_DATEN from XML")
klb_data_found = True
xklb_array, yklb_array = readMacrocontur(root)
i = 0
for x,y in zip(xklb_array,yklb_array):
  print(i, " ",x,y)
  i = i+1

if xklb_array.size > 0 and yklb_array.size > 0:
  print("After change position")
  klb_data_found = True
  xklb_array,yklb_array = changeContur(xklb_array,yklb_array, xklb_array.size-1, yklb_array.size-1)
  i = 0
  for x,y in zip(xklb_array,yklb_array):
    print(i, " ",x,y)
    i = i+1
else:
    print("Keine KLB_DATEN im XML vorhanden")
    klb_data_found = False


In [ ]:
nom_diameter = 0.0
total_length = 0.0

top_land = 0.0
top_land_diameter = 0.0

groove1 = 0.0
groove1_diameter = 0.0

land2 = 0.0
groove2 = 0.0

land2_diameter = 0.0
groove2_diameter = 0.0

land3 = 0.0
groove3 = 0.0

land3_diameter = 0.0
groove3_diameter = 0.0

land4 = 0.0
groove4 = 0.0

land4_diameter = 0.0
groove4_diameter = 0.0

land5 = 0.0
groove5 = 0.0

land5_diameter = 0.0
groove5_diameter = 0.0

skirt_length = 0.0

if root:
  yklb_values = []
  xklb_values = []
  for element in root.findall(".//HAUPTABMESSUNGEN"):
    for child in element:
      if child.tag == 'ANZAHL_NUTEN':
        print("Anzahl Nuten: ",child.text)
      elif child.tag == 'HOEHE':
        print("Hoehe: ",child.text);
        total_length = float(child.text)

      elif child.tag == 'FEUERSTEG_LAENGE':
        print("----------------------------------------")
        print("FEUERSTEG_LAENGE = top_land_length",child.text)
        top_land = float(child.text)
      elif child.tag == 'FEUERSTEG_DURCHMESSER':
        print("FEUERSTEG_DURCHMESSER",child.text)
        top_land_diameter= float(child.text)

      elif child.tag == 'RINGSTEG4_LAENGE':
        print("----------------------------------------")
        land5 = float(child.text)
        print("RINGSTEG5_LAENGE",child.text)
      elif child.tag == 'RINGSTEG4_DURCHMESSER':
        print("RINGSTEG5_DURCHMESSER",child.text)
        land5_diameter = float(child.text)

      elif child.tag == 'RINGSTEG3_LAENGE':
        print("----------------------------------------")
        land4 = float(child.text)
        print("RINGSTEG4_LAENGE",child.text)
      elif child.tag == 'RINGSTEG3_DURCHMESSER':
        print("RINGSTEG4_DURCHMESSER",child.text)
        land4_diameter = float(child.text)

      elif child.tag == 'RINGSTEG2_LAENGE':
        print("----------------------------------------")
        land3 = float(child.text)
        print("RINGSTEG3_LAENGE",child.text)
      elif child.tag == 'RINGSTEG2_DURCHMESSER':
        print("RINGSTEG3_DURCHMESSER",child.text)
        land3_diameter = float(child.text)

      elif child.tag == 'RINGSTEG1_LAENGE':
        print("----------------------------------------")
        land2 = float(child.text)
        print("RINGSTEG2_LAENGE",child.text)
      elif child.tag == 'RINGSTEG1_DURCHMESSER':
        print("RINGSTEG2_DURCHMESSER",child.text)
        land2_diameter = float(child.text)

      elif child.tag == 'NUT5_LAENGE':
        print("----------------------------------------")
        print("NUT5_LAENGE",child.text)
        groove5 = float(child.text)
      elif child.tag == 'NUT5_DURCHMESSER':
        print("NUT5_DURCHMESSER",child.text)
        groove5_diameter = float(child.text)

      elif child.tag == 'NUT4_LAENGE':
        print("----------------------------------------")
        print("NUT4_LAENGE",child.text)
        groove4 = float(child.text)
      elif child.tag == 'NUT4_DURCHMESSER':
        print("NUT4_DURCHMESSER",child.text)
        groove4_diameter = float(child.text)

      elif child.tag == 'NUT3_LAENGE':
        print("----------------------------------------")
        print("NUT3_LAENGE",child.text)
        groove3 = float(child.text)
      elif child.tag == 'NUT3_DURCHMESSER':
        print("NUT3_DURCHMESSER",child.text)
        groove3_diameter = float(child.text)

      elif child.tag == 'NUT2_LAENGE':
        print("----------------------------------------")
        print("NUT2_LAENGE",child.text)
        groove2 = float(child.text)
      elif child.tag == 'NUT2_DURCHMESSER':
        print("NUT2_DURCHMESSER",child.text)
        groove2_diameter = float(child.text)

      elif child.tag == 'NUT1_LAENGE':
        print("----------------------------------------")
        print("NUT1_LAENGE",child.text)
        groove1 = float(child.text)
      elif child.tag == 'NUT1_DURCHMESSER':
        print("NUT1_DURCHMESSER",child.text)
        groove1_diameter = float(child.text)

      elif child.tag == 'SCHAFTDURCHMESSER':
        print("SCHAFTDURCHMESSER",child.text)
        nom_diameter = float(child.text)
      elif child.tag == 'SCHAFTLAENGE':
        print("----------------------------------------")
        print("SCHAFTLAENGE",child.text)
        skirt_length= float(child.text)
      elif child.tag == 'DURCHMESSER':
        print("DURCHMESSER",child.text)


In [ ]:
klb_data_found = False
if klb_data_found == False:
  print("keine KLB Daten vorhanden")
  nNuten = 100
  # Erstelle ein leeres Array mit der gewünschten Größe
  xklb_array = np.empty(nNuten)
  yklb_array = np.empty(nNuten)
  i = 0
  ixy = 0
  for i in range(xklb_array.size):
    xklb_array[i] = float('nan')
    yklb_array[i] = float('nan')

  # Start skirt links unten
  print("***start skirt***")
  xklb_array[ixy] = 0 #nom_diameter/2
  yklb_array[ixy] = 0
  print("1) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1

  # Start skirt hori. nach rechts unten
  xklb_array[ixy] = nom_diameter/2
  yklb_array[ixy] = 0 #skirt_length
  print("2) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1

  # Skirt vertikale rechts unten nach rechts skirt len.
  xklb_array[ixy] = nom_diameter/2
  yklb_array[ixy] = skirt_length
  print("3) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1

  # Nut
  if groove5 > 0.0:
    print("***groove5***")
    xklb_array[ixy] = groove5_diameter/2
    yklb_array[ixy] = skirt_length
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove5_diameter/2
    yklb_array[ixy] = skirt_length+groove5
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land5 > 0.0:
    print("***land5***")
    xklb_array[ixy] = land5_diameter/2
    yklb_array[ixy] = skirt_length+groove5
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land5_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if groove4 > 0.0:
    print("***groove4***")
    xklb_array[ixy] = groove4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land4 > 0.0:
    print("***land4***")
    xklb_array[ixy] = land4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land4_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if groove3 > 0.0:
    print("***groove3***")
    xklb_array[ixy] = groove3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land3 > 0.0:
    print("***land3***")
    xklb_array[ixy] = land3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land3_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1


  if groove2 > 0.0:
    print("***groove2***")
    xklb_array[ixy] = groove2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if land2 > 0.0:
    print("***land2***")
    xklb_array[ixy] = land2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = land2_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if groove1 > 0.0:
    print("***groove1***")
    xklb_array[ixy] = groove1_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2
    print("1) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    xklb_array[ixy] = groove1_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1
    print("2) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  if top_land > 0.0:
    print("***top_land (Feuersteg)***")
    xklb_array[ixy] = top_land_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1
    print("3) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

    #Feuersteg
    xklb_array[ixy] = top_land_diameter/2
    yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1+top_land
    print("4) ", xklb_array[ixy],yklb_array[ixy])
    ixy = ixy+1

  #Ende Kolben Feuersteg horizontal
  print("Kolben Feuersteg Horizontal")
  xklb_array[ixy] = 0
  yklb_array[ixy] = skirt_length+groove5+land5+groove4+land4+groove3+land3+groove2+land2+groove1+top_land
  print("1) ", xklb_array[ixy],yklb_array[ixy])
  ixy = ixy+1
  print("********************************************")

  # Reihenfolge der xklb_array und yklb_array umdrehen
  xklb_array = xklb_array[::-1]
  yklb_array = yklb_array[::-1]

  # Entfernen von NaN-Werten aus den Arrays
  valid_indices = ~np.isnan(xklb_array) & ~np.isnan(yklb_array)
  xklb_array = xklb_array[valid_indices]
  yklb_array = yklb_array[valid_indices]

# Test Ausgabe
i = 0
for x,y in zip(xklb_array,yklb_array):
    print(i, " ",x,y)
    i = i+1
# Test Plot
# plt.plot(xklb_array, yklb_array, marker='.', linestyle='-')



# Plot Macrocontur

In [ ]:
# Plot erstellen
plt.figure(figsize=(10, 8))  # Set the figure size (width, height) in inches
plt.xlim(0, int(nom_diameter*1.5))
plt.ylim(0, int(total_length*1.5))
plt.plot(xklb_array, yklb_array, marker='.', linestyle='-')

hlinewidth=1.5
hxmin=1 #nom_diameter/10
hxmax= nom_diameter/2  -1#nom_diameter/2 +4
# Feuersteg Höhe
if top_land > 0:
  top_land_hight_absolute = skirt_length+groove5+groove4+groove3+groove2+groove1+land5+land4+land3+land2+top_land
  plt.hlines(y=top_land_hight_absolute, xmin=hxmin, xmax=hxmax, colors='darkgreen', linestyles='--', linewidth=hlinewidth, label='a')
  plt.text(hxmax+2, top_land_hight_absolute, "Top Land", color='darkgreen', fontsize=6)
  # untere kante Feuersteg
  plt.hlines(y=top_land_hight_absolute - top_land, xmin=hxmin, xmax=hxmax, colors='darkgreen', linestyles='--', linewidth=hlinewidth, label='a')
  plt.text(hxmax+2, top_land_hight_absolute- top_land, "Top Land", color='darkgreen', fontsize=6)

# if groove1 > 0:
#  plt.hlines(y=skirt_length+groove5+groove4+groove3+groove2+groove1+land5+land4+land3+land2, xmin=hxmin, xmax=hxmax, colors='darkgreen', linestyles='--', linewidth=hlinewidth,label='a')

if land2 > 0:
  land2_height_absolute = skirt_length+groove5+groove4+groove3+groove2+land5+land4+land3+land2
  plt.hlines(y=land2_height_absolute, xmin=hxmin, xmax=hxmax, colors='gold', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land2_height_absolute, "2nd land", color='gold', fontsize=6)
  plt.hlines(y=land2_height_absolute-land2, xmin=hxmin, xmax=hxmax, colors='gold', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land2_height_absolute-land2, "2nd land", color='gold', fontsize=6)

if land3 > 0:
  land3_height_absolute = skirt_length+groove5+land5+groove4+land4+groove3+land3
  plt.hlines(y=land3_height_absolute, xmin=hxmin, xmax=hxmax, colors='purple', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land3_height_absolute, "3nd land", color='purple', fontsize=6)
  plt.hlines(y=land3_height_absolute-land3, xmin=hxmin, xmax=hxmax, colors='purple', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land3_height_absolute-land3, "3nd land", color='purple', fontsize=6)

if land4 > 0:
  land4_height_absolute = skirt_length+groove5+land5+groove4+land4
  plt.hlines(y=land4_height_absolute, xmin=hxmin, xmax=hxmax, colors='darkred', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land4_height_absolute, "4nd land", color='darkred', fontsize=6)
  plt.hlines(y=land4_height_absolute-land4, xmin=hxmin, xmax=hxmax, colors='darkred', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land4_height_absolute-land4, "4nd land", color='darkred', fontsize=6)

if land5 > 0:
  land5_height_absolute = skirt_length+groove5+land5
  plt.hlines(y=land5_height_absolute, xmin=hxmin, xmax=hxmax, colors='gray', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land5_height_absolute, "5nd land", color='darkred', fontsize=6)
  plt.hlines(y=land5_height_absolute-land5, xmin=hxmin, xmax=hxmax, colors='gray', linestyles='--',linewidth=hlinewidth, label='b')
  plt.text(hxmax+2, land5_height_absolute-land5, "5nd land", color='darkred', fontsize=6)

# Skirt
if skirt_length > 0:
  skirt_length_absolute = skirt_length
  plt.hlines(y=skirt_length_absolute, xmin=hxmin, xmax=hxmax, colors='blue', linestyles='--',linewidth=hlinewidth, label='e')
  plt.text(hxmax+2, skirt_length_absolute, "Skirt", color='blue', fontsize=6)
  plt.hlines(y=0, xmin=hxmin, xmax=hxmax, colors='blue', linestyles='--',linewidth=hlinewidth+1, label='f')
  plt.text(hxmax+2, 0, "Skirt", color='blue', fontsize=6)

# Ausgabe der Formkurven-Abschnitte
hxmin= nom_diameter / 2 + 12
hxmax=nom_diameter + 20
section = 1
for ll in level_limit:
  plt.hlines(y=ll, xmin=hxmin, xmax=hxmax, colors='black', linestyles='-',linewidth=hlinewidth)
  if (section % 2) > 0:
    plt.text(nom_diameter / 2 + 12, ll +1, "Section " + str(section), color='black', fontsize=6)
  section = section +1



plt.xlabel("X [mm]")
plt.ylabel("Y [mm]")
plt.title(fn)
plt.grid(True)
# plt.legend()
plt.show()


In [ ]:
# Plot erstellen
plt.figure(figsize=(10, 8))  # Set the figure size (width, height) in inches
plt.xlim(0, int(nom_diameter*1.5))
plt.ylim(0, int(total_length*1.5))
plt.plot(xa[:1], ya[:1], marker='.', linestyle='-')
print( xa[:1], ya[:1])

